In [7]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    tools = [TavilySearchResults()]
    graph_builder = StateGraph(State)
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.set_entry_point("chatbot")

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")

    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)

    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values"
    )

    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break

    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
台湾観光について検索結果を教えて

[{"url": "https://www.his-j.com/sightseeing/kaigai/asia/taiwan/", "content": "総合旅行サイトHIS\n 海外旅行\n 海外観光情報\n アジア観光\n 台湾観光\n\n## 台湾観光情報\n\nグルメ、ショッピング、足つぼマッサージなど魅力いっぱいの国。食べ歩きから一流レストランまで楽しめる台北。台湾スイーツ発祥の地といわれる台中や、海鮮料理が魅力の高雄。レトロな街並みが楽しめる台南と、北から南まで楽しさが詰まっています。\n\n おすすめ観光地\n  + -スポットランキング\n  + -観光地情報\n 台湾の旅行選びのポイント\n  + -台湾の人気グルメ\n  + -台湾人気夜市3選\n  + -人気のノスタルジック老街3選\n おすすめホテル\n おすすめオプショナルツアー\n 観光基本情報\n 関連リンク\n\n### 台湾でのおすすめ観光 スポットランキング\n\n#### 台北101台北/信義 [...] 台湾有数のパワースポット「龍から入って虎から出る」  \n複数の中国建築が建つ蓮池潭の最南端。湖上にそびえる二つの塔が龍虎塔です。ジグザグの橋を歩き進めた先に待ち受けるのは、大きく口を開いた龍と虎。龍塔と虎塔はそれぞれ7層で、最上部まで上ると湖が一望できます。高雄を訪れるなら、厄落としの意味を持つ龍と虎の体を抜けて、幸運を呼び込んでみてはいかがでしょうか。龍から入って虎から出ることで、厄除けできるといわれています。\n\n### 台湾旅行の良いところ！\n\n### 台湾でのおすすめ観光地情報\n\n#### 台中公園\n\n#### 逢甲夜市\n\n台中\n\n#### 高美湿地\n\n台中\n\n#### 美麗島駅\n\n#### 花蓮\n\n台東\n\n#### 日月潭（リーユエタン）\n\n台湾最大の淡水湖である日月潭は、美しい湖面の色合いが人々の心をとらえます。豊かな自然美の中で、心安らぐリラクゼーションタイムが過ごせます。\n\n台中\n\n#### 故宮博物院\n\n世界四大博物館のひとつで、世界一の中国美術工芸コレクションを誇ります。必見の文物を見逃さないた